In [36]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [ ]:
# Warning, the sendtofile doesn't work when connected to the fan of sensiron

In [44]:
%sendtofile --source configSCD30.txt

No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands

In [33]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [38]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py
# devices SCD30,SCD41

Sent 75 lines (2549 bytes) to stdmqttas.py.
Sent 39 lines (1151 bytes) to utils.py.


In [39]:
%serialconnect
# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

# !!! The redo for different connections isn't working so put the one 
# that is active into connection0

Sent 731 lines (27364 bytes) to mqtt_as.py.


In [10]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [40]:
%sendtofile main.py

import time, sys, machine, struct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask
from stdmqttas import clientsingleton, WatchDogtask, cdelimeter
from stdmqttas import shortmac, topicstem, itertools_count
from stdmqttas import pinled, pinledOnvalue, flashpinled
from utils import crc8

import network
network.WLAN().active(0)  # disable the connection at startup
print("Initial flashing of ledpin")
flashpinled(5, 300, 300)

i2c = machine.I2C(1, scl=machine.Pin(int(fconfig["pinscl"])), sda=machine.Pin(int(fconfig["pinsda"])))

topicscd30 = topicstem+"/scd30"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"
sensorposition = fconfig.get("position", "")
print("topicstem: '%s'" % topicstem)
print("sensorposition: '%s'" % sensorposition)

async def fanonwhenconnectedtask():
    pinfan = machine.Pin(int(fconfig["pinfan"]), machine.Pin.OUT)
    while True:
        pinfan.value(clientsingleton and clientsingleton[0].isconnected())
        await asyncio.sleep_ms(500)

async def SCD30task():
    try:
        i2c.writeto(0x61, bytes([0x02, 0x04]))   # soft reset
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x00, 0x10, 0x00, 0x00, crc8([0x00, 0x00])]))   # continuous measurement
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x46, 0x00, 0x00, 0x02, crc8([0x00, 0x02])]))   # 2s measurement interval
        await asyncio.sleep_ms(40)
        for n in itertools_count():
            y = i2c.writeto(0x61, bytes([0x02, 0x02]))  # data ready?
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x61, 2)    # ready is 1
            if d == bytes([0x00, 0x01]):
                y = i2c.writeto(0x61, bytes([0x03, 0x00]))  # read data
                await asyncio.sleep_ms(4)
                x = i2c.readfrom(0x61, 18)
                #crc8(x[:2]) == x[2]  etc for every 3 bytes
                co2, temp, humid = struct.unpack(">fff", bytes([x[0],x[1],x[3],x[4], x[6],x[7],x[9],x[10], x[12],x[13],x[15],x[16]]))
                print(n, topicscd30, co2, temp, humid)
                if clientsingleton and clientsingleton[0].isconnected():
                    await clientsingleton[0].publish(topicscd30, "%f %f %f %s"%(co2, temp, humid, sensorposition))
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD30taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(SCD30task())
            
# mosquitto_pub -h mqtt.local -t "esp32/7AEDB8/cmd" -m "print('hi there')"   
async def callbackcmdtask(topicreply, codemsg):
    print("executing", [codemsg])
    try:
        exec(codemsg)
        await clientsingleton[0].publish(topicreply, "1")
    except Exception as e:
        print(e)
        await clientsingleton[0].publish(topicreply+'/exception', str(e))
    return

def callbackcmd(topic, msg, retained):
    print("Received message", (topic, msg))
    if topic == topiccmd: 
        aloop.create_task(callbackcmdtask(topicreply, msg))

Nreconnections = 0
async def onconnecttask(client):
    global Nreconnections
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.publish(topicerror, "reconnections%d"%Nreconnections if Nreconnections else "", retain=True)
    await client.subscribe(topiccmd, 1)
    Nreconnections += 1

config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)

deviceslist = fconfig.get("devices", "SCD30").split(cdelimeter)

aloop = asyncio.get_event_loop()
aloop.create_task(fanonwhenconnectedtask())
aloop.create_task(mqttconnecttask())
if "SCD30" in deviceslist:
    aloop.create_task(SCD30task())

aloop.run_forever()

if "wdtsecs" in fconfig:
    aloop.create_task(WatchDogtask(float(fconfig["wdtsecs"])))
aloop.run_forever()



Sent 103 lines (4015 bytes) to main.py.


In [31]:
%ls

Listing directory '/'.
      139    boot.py
      284    config.txt
     5679    main.py
    23172    mqtt_as.py
     2679    stdmqttas.py
      969    utils.py
